# ln._utils - Core Utility Functions

Essential utilities for lionpride operations:

**Time Management:**
- **now_utc()**: Current UTC timestamp

**Async Path Operations:**
- **acreate_path()**: Async file path creation with timestamps, random hashes, and timeouts

**Data Organization:**
- **get_bins()**: Bin packing for strings by cumulative length

**Dynamic Imports:**
- **import_module()**: Flexible module importing by path
- **is_import_installed()**: Check package availability

In [1]:
import tempfile

from lionpride.ln._utils import (
    acreate_path,
    get_bins,
    import_module,
    is_import_installed,
    now_utc,
)

## 1. UTC Timestamps with now_utc()

Simple utility to get timezone-aware UTC datetime.

In [2]:
# Get current UTC time
timestamp = now_utc()
print(f"Current UTC: {timestamp}")
print(f"Timezone: {timestamp.tzinfo}")
print(f"ISO format: {timestamp.isoformat()}")

Current UTC: 2025-11-23 18:55:18.756923+00:00
Timezone: UTC
ISO format: 2025-11-23T18:55:18.756923+00:00


In [3]:
# Use for consistent timestamps
import time

t1 = now_utc()
time.sleep(0.1)
t2 = now_utc()

print(f"Time elapsed: {(t2 - t1).total_seconds():.2f}s")
print(f"Both have UTC timezone: {t1.tzinfo == t2.tzinfo}")

Time elapsed: 0.10s
Both have UTC timezone: True


## 2. Async Path Creation with acreate_path()

Flexible async file path generation with:
- Subdirectory support via `/` in filename
- Optional timestamps
- Random hash suffixes
- Timeout protection

In [4]:
# Basic usage - simple filename
with tempfile.TemporaryDirectory() as tmpdir:
    path = await acreate_path(tmpdir, "test", "txt")
    print(f"Created: {path}")
    print(f"Type: {type(path).__name__}")
    print(f"Exists: {await path.parent.exists()}")

Created: /var/folders/5p/rcbw097d29j3s2qt861tsjfh0000gn/T/tmpph29iiqo/test.txt
Type: Path
Exists: True


In [5]:
# Subdirectory support - filename with slashes
with tempfile.TemporaryDirectory() as tmpdir:
    path = await acreate_path(tmpdir, "logs/2025/report.txt")
    print(f"Path: {path}")
    print(f"Parent exists: {await path.parent.exists()}")
    print(f"Parent name: {path.parent.name}")

Path: /var/folders/5p/rcbw097d29j3s2qt861tsjfh0000gn/T/tmpspeg0rsv/logs/2025/report.txt
Parent exists: True
Parent name: 2025


In [6]:
# Timestamp options
with tempfile.TemporaryDirectory() as tmpdir:
    # Timestamp suffix (default)
    path1 = await acreate_path(tmpdir, "report", "txt", timestamp=True)
    print(f"Suffix: {path1.name}")

    # Timestamp prefix
    path2 = await acreate_path(tmpdir, "report", "txt", timestamp=True, time_prefix=True)
    print(f"Prefix: {path2.name}")

    # Custom timestamp format
    path3 = await acreate_path(tmpdir, "report", "txt", timestamp=True, timestamp_format="%Y-%m-%d")
    print(f"Custom format: {path3.name}")

Suffix: report_20251123135518.txt
Prefix: 20251123135518_report.txt
Custom format: report_2025-11-23.txt


In [7]:
# Random hash suffix for uniqueness
with tempfile.TemporaryDirectory() as tmpdir:
    path1 = await acreate_path(tmpdir, "session", "log", random_hash_digits=8)
    path2 = await acreate_path(tmpdir, "session", "log", random_hash_digits=8)

    print(f"Path 1: {path1.name}")
    print(f"Path 2: {path2.name}")
    print(f"Unique: {path1.name != path2.name}")

Path 1: session-b4b95fd6.log
Path 2: session-0d122691.log
Unique: True


In [8]:
# Combining timestamp + random hash
with tempfile.TemporaryDirectory() as tmpdir:
    path = await acreate_path(
        tmpdir,
        "output/results.csv",
        timestamp=True,
        random_hash_digits=6,
        timestamp_format="%Y%m%d",
    )
    print(f"Full path: {path}")
    print(f"Filename: {path.name}")

Full path: /var/folders/5p/rcbw097d29j3s2qt861tsjfh0000gn/T/tmpjvk82xxd/output/results_20251123-1ec64d.csv
Filename: results_20251123-1ec64d.csv


In [9]:
# File existence control
with tempfile.TemporaryDirectory() as tmpdir:
    path = await acreate_path(tmpdir, "unique", "txt")

    # Create the file
    await path.touch()

    # Try creating again with file_exist_ok=False (default)
    try:
        await acreate_path(tmpdir, "unique.txt", file_exist_ok=False)
    except FileExistsError as e:
        print(f"✓ File exists error caught: {type(e).__name__}")

    # Allow existing file
    path2 = await acreate_path(tmpdir, "unique.txt", file_exist_ok=True)
    print(f"✓ file_exist_ok=True allows existing: {path == path2}")

✓ File exists error caught: FileExistsError
✓ file_exist_ok=True allows existing: True


In [10]:
# Timeout protection for slow I/O
with tempfile.TemporaryDirectory() as tmpdir:
    # Normal operation completes quickly
    path = await acreate_path(tmpdir, "fast", "txt", timeout=1.0)
    print(f"✓ Completed within timeout: {path.name}")

    # Note: Actual timeout demonstration would require slow filesystem
    # In practice, typical operations complete in <100ms

✓ Completed within timeout: fast.txt


## 3. Bin Packing with get_bins()

Organize string indices into bins by cumulative length - useful for batching within token limits.

In [11]:
# Basic binning
strings = ["short", "a bit longer", "tiny", "this is a much longer string", "mid"]
bins = get_bins(strings, upper=20)

print("Strings:", strings)
print("\nBins (max 20 chars cumulative):\n")
for i, bin_indices in enumerate(bins):
    bin_strings = [strings[idx] for idx in bin_indices]
    total_len = sum(len(s) for s in bin_strings)
    print(f"  Bin {i}: indices {bin_indices}")
    print(f"    Strings: {bin_strings}")
    print(f"    Total length: {total_len}")

Strings: ['short', 'a bit longer', 'tiny', 'this is a much longer string', 'mid']

Bins (max 20 chars cumulative):

  Bin 0: indices [0, 1]
    Strings: ['short', 'a bit longer']
    Total length: 17
  Bin 1: indices [2]
    Strings: ['tiny']
    Total length: 4
  Bin 2: indices [3]
    Strings: ['this is a much longer string']
    Total length: 28
  Bin 3: indices [4]
    Strings: ['mid']
    Total length: 3


In [12]:
# Token limit simulation
messages = [
    "Hello",
    "How are you?",
    "I'm doing great!",
    "What's the weather like?",
    "Sunny",
    "Perfect!",
]

# Bin by 30-character chunks
bins = get_bins(messages, upper=30)

print("Organizing messages into batches:\n")
for i, bin_indices in enumerate(bins):
    batch = [messages[idx] for idx in bin_indices]
    print(f"Batch {i + 1}: {batch}")

Organizing messages into batches:

Batch 1: ['Hello', 'How are you?']
Batch 2: ["I'm doing great!"]
Batch 3: ["What's the weather like?", 'Sunny']
Batch 4: ['Perfect!']


In [13]:
# Edge case: String longer than limit
items = ["a", "this exceeds the limit by far", "b", "c"]
bins = get_bins(items, upper=10)

print("When item > limit, it gets its own bin:\n")
for i, bin_indices in enumerate(bins):
    print(f"Bin {i}: {[items[idx] for idx in bin_indices]}")

When item > limit, it gets its own bin:

Bin 0: ['a']
Bin 1: ['this exceeds the limit by far']
Bin 2: ['b', 'c']


## 4. Dynamic Imports with import_module()

Import modules, packages, and specific objects dynamically by path.

In [14]:
# Import entire module
json_module = import_module("json")
print(f"Imported: {json_module.__name__}")
print(f"Has dumps: {hasattr(json_module, 'dumps')}")

Imported: json
Has dumps: True


In [15]:
# Import specific object from module
PathClass = import_module("pathlib", import_name="Path")
print(f"Imported: {PathClass.__name__}")
print(f"Type: {type(PathClass)}")

# Use it
p = PathClass("/tmp/test")
print(f"Created path: {p}")

Imported: Path
Type: <class 'type'>
Created path: /tmp/test


In [16]:
# Import multiple objects at once
dumps, loads = import_module("json", import_name=["dumps", "loads"])
print(f"Imported: {dumps.__name__}, {loads.__name__}")

# Use them
data = {"test": 123}
json_str = dumps(data)
restored = loads(json_str)
print(f"Roundtrip: {data} -> '{json_str}' -> {restored}")

Imported: dumps, loads
Roundtrip: {'test': 123} -> '{"test": 123}' -> {'test': 123}


In [17]:
# Import from nested package
Element = import_module("lionpride.core", import_name="Element")
print(f"Imported: {Element.__name__}")

# Create instance
elem = Element()
print(f"Created: {type(elem).__name__} with ID {elem.id}")

Imported: Element
Created: Element with ID 359e9461-73d5-4d91-854a-d2ece074ccd0


In [18]:
# Error handling for missing modules
try:
    import_module("nonexistent_package_xyz")
except ImportError as e:
    print(f"✓ Import error caught: {str(e)[:50]}...")

✓ Import error caught: Failed to import module nonexistent_package_xyz: N...


## 5. Package Detection with is_import_installed()

Check if packages are available before attempting imports.

In [19]:
# Check standard library
print(f"json installed: {is_import_installed('json')}")
print(f"pathlib installed: {is_import_installed('pathlib')}")
print(f"asyncio installed: {is_import_installed('asyncio')}")

json installed: True
pathlib installed: True
asyncio installed: True


In [20]:
# Check third-party packages
print(f"pydantic installed: {is_import_installed('pydantic')}")
print(f"anyio installed: {is_import_installed('anyio')}")

pydantic installed: True
anyio installed: True


In [21]:
# Check non-existent package
print(f"fake_package_xyz installed: {is_import_installed('fake_package_xyz')}")

fake_package_xyz installed: False


In [22]:
# Conditional import pattern
if is_import_installed("pydantic"):
    print("✓ Pydantic available, using BaseModel")
else:
    print("⚠ Pydantic not available, using alternative")

✓ Pydantic available, using BaseModel


## 6. Real-World Use Cases

Combining utilities for common patterns.

In [23]:
# Timestamped log file creation
with tempfile.TemporaryDirectory() as tmpdir:
    log_path = await acreate_path(
        tmpdir,
        "sessions/session",
        "log",
        timestamp=True,
        random_hash_digits=4,
        timestamp_format="%Y%m%d_%H%M%S",
    )

    # Write timestamped entry
    await log_path.write_text(f"[{now_utc().isoformat()}] Session started\n")

    content = await log_path.read_text()
    print(f"Log file: {log_path.name}")
    print(f"Content: {content.strip()}")

Log file: session_20251123_135518-a8c1.log
Content: [2025-11-23T18:55:18.936501+00:00] Session started


In [24]:
# Batch message processing with token limits
messages = [
    "User: Hello",
    "Assistant: Hi there!",
    "User: How's the weather?",
    "Assistant: Sunny and warm!",
    "User: Great for a walk",
    "Assistant: Absolutely!",
]

# Organize into batches (simulate 50-char token limit)
bins = get_bins(messages, upper=50)

print("Processing message batches:\n")
for i, bin_indices in enumerate(bins):
    batch = [messages[idx] for idx in bin_indices]
    timestamp = now_utc()
    print(f"[{timestamp.strftime('%H:%M:%S')}] Batch {i + 1}: {len(batch)} messages")
    for msg in batch:
        print(f"  - {msg}")

Processing message batches:

[18:55:18] Batch 1: 2 messages
  - User: Hello
  - Assistant: Hi there!
[18:55:18] Batch 2: 1 messages
  - User: How's the weather?
[18:55:18] Batch 3: 2 messages
  - Assistant: Sunny and warm!
  - User: Great for a walk
[18:55:18] Batch 4: 1 messages
  - Assistant: Absolutely!


In [25]:
# Optional feature loading with graceful fallback
def get_serializer():
    """Get best available JSON serializer."""
    if is_import_installed("orjson"):
        orjson = import_module("orjson")
        return "orjson", orjson.dumps, orjson.loads
    else:
        json = import_module("json")
        return "json", json.dumps, json.loads


name, dumps, loads = get_serializer()
print(f"Using serializer: {name}")

# Test it
data = {"timestamp": now_utc().isoformat(), "status": "ok"}
serialized = dumps(data)
print(f"Serialized: {serialized}")

Using serializer: orjson
Serialized: b'{"timestamp":"2025-11-23T18:55:18.943825+00:00","status":"ok"}'


## Summary Checklist

**Time Utilities:**
- ✅ `now_utc()` for consistent UTC timestamps
- ✅ Timezone-aware datetime objects

**Path Management:**
- ✅ `acreate_path()` for async file path creation
- ✅ Subdirectory support via `/` in filename
- ✅ Optional timestamps (prefix/suffix, custom format)
- ✅ Random hash suffixes for uniqueness
- ✅ File existence control
- ✅ Timeout protection for I/O operations

**Data Organization:**
- ✅ `get_bins()` for batching strings by cumulative length
- ✅ Useful for token limits, message batching

**Dynamic Imports:**
- ✅ `import_module()` for runtime imports
- ✅ Import entire modules or specific objects
- ✅ Multiple object imports in one call
- ✅ `is_import_installed()` for availability checks
- ✅ Graceful fallbacks for optional dependencies

**Common Patterns:**
- ✅ Timestamped log files with unique IDs
- ✅ Batch processing with size limits
- ✅ Optional feature loading with fallbacks